In [1]:
# Setup environment
import os
import glob
import subprocess
import sys
import time
from scipy.io import netcdf as nc

import matplotlib.pyplot as plt
from os.path import expanduser

# Define output path
home = expanduser("~")
output_dir = os.path.join(home,'scratch')  # main output directory.
os.makedirs(os.path.dirname(output_dir), exist_ok=True)
print(output_dir)

/Users/shawnserbin/scratch


In [2]:
# Grab latest CTSM docker container from Docker hub
source = "serbinsh/ctsm_containers"
docker_container_tag = "ctsm-fates_next_api-fates_sci.1.23.0_api.7.1.0"
container_name = ''.join([source,":",docker_container_tag])
print(container_name)
docker_pull_command = ''.join(["docker pull ",source,":",docker_container_tag])
os.system(docker_pull_command)

# you should see docker pull progress in the Jupyter shell

serbinsh/ctsm_containers:ctsm-fates_next_api-fates_sci.1.23.0_api.7.1.0


0

In [5]:
# Define run parameters for CTSM
start_year = '1900-01-01'  # you need to place this in the single quotations
num_years = 120
met_start = 2008
met_end = 2016
run_type = "startup"
hostname = "modex"
user = "clmuser"

# Define input/output paths (host:container)
cesm_input_data_paths = "~Data/cesm_input_data:/data"
output_path = ''.join([output_dir,":/ctsm_output"])
print(output_path)

# Define script to run
host_script_dir = "/Users/sserbin/Data/GitHub/ctsm_containers/ctsm_run_scripts:/ctsm_run_scripts"
ctsm_case_build_script = "/ctsm_run_scripts/create_case_ctsmfates_1pt_example_PA-SLZ.sh"


/Users/shawnserbin/scratch:/ctsm_output


In [6]:
# Define case build command
run_command = ''.join(["docker run --hostname=",hostname," --user ",user," -v ", host_script_dir," -v ",cesm_input_data_paths, " -v ", output_path, " ", container_name, " ", ctsm_case_build_script, " ", "-sy=","'",start_year.__str__(),"'", " ", "-ny=",num_years.__str__()," ","-rt=",run_type," ","-mets=",met_start.__str__()," ","-mete=",met_end.__str__()])
print(run_command)

docker run --hostname=modex --user clmuser -v /Users/sserbin/Data/GitHub/ctsm_containers/ctsm_run_scripts:/ctsm_run_scripts -v ~Data/cesm_input_data:/data -v /Users/shawnserbin/scratch:/ctsm_output serbinsh/ctsm_containers:ctsm-fates_next_api-fates_sci.1.23.0_api.7.1.0 /ctsm_run_scripts/create_case_ctsmfates_1pt_example_PA-SLZ.sh -sy='1900-01-01' -ny=120 -rt=startup -mets=2008 -mete=2016


In [ ]:
# Build the case
start = time.time()
# run the docker container
os.system(run_command)
# get runtime
end = time.time()
#Print runtime
print(end - start)